In [5]:
#  unsloth_env enviorment
from unsloth import FastLanguageModel
from torch.amp import autocast
from sklearn.model_selection import train_test_split
import gc
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


ModuleNotFoundError: No module named 'sklearn'

: 

In [2]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

#device = "cuda" if torch.cuda.is_available() else "cpu"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    #dtype =torch.float32
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


==((====))==  Unsloth 2024.10.3: Fast Llama patching. Transformers = 4.45.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2080 Ti. Max memory: 10.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`


In [3]:
torch.cuda.empty_cache()
gc.collect()

0

In [4]:
#model.save_pretrained("pretrain_model/unsloth/Meta-Llama-3.1-8B-bnb-4bit") # Local saving
#tokenizer.save_pretrained("pretrain_tokenizer/unsloth/Meta-Llama-3.1-8B-bnb-4bit")

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
import pandas as pd
Question_list = []
def prepare_cornell_dataset(file_path):
    # 读取电影台词数据
    lines = {}
    with open(file_path, 'r', encoding='iso-8859-1') as f:
        for line in f:
            parts = line.strip().split(" +++$+++ ")
            lines[parts[0]] = parts[-1]  # 提取 lineID 和 句子内容

    # 读取对话数据
    conversations = []
    with open('/home/kaia/BasicTR/cornell movie-dialogs corpus/movie_conversations.txt', 'r', encoding='iso-8859-1') as f:
        for line in f:
            parts = line.strip().split(" +++$+++ ")
            # 将字符串的 list 形式转换为真正的列表
            line_ids = eval(parts[-1])
            for i in range(len(line_ids) - 1):
                conversations.append((lines[line_ids[i]], lines[line_ids[i + 1]]))

    # 将对话数据保存为一个文本文件，供微调使用
    with open("prepared_cornell_dialogs.txt", "w") as out_f:
        for q, a in conversations:
            out_f.write(f"{q}\t{a}\n")  # 用 tab 分隔问题和回答
            #print(f'question: {q}')
            #print(f'answer: {a}')
            Question_list.append(a)

# 调用数据准备函数
prepare_cornell_dataset('/home/kaia/BasicTR/cornell movie-dialogs corpus/movie_lines.txt')
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

In [6]:
torch.cuda.empty_cache()
gc.collect()

0

In [8]:

'''
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        print(text)
        break
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
'''

'\nalpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{}\n\n### Input:\n{}\n\n### Response:\n{}"""\n\nEOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN\ndef formatting_prompts_func(examples):\n    instructions = examples["instruction"]\n    inputs       = examples["input"]\n    outputs      = examples["output"]\n    texts = []\n    for instruction, input, output in zip(instructions, inputs, outputs):\n        # Must add EOS_TOKEN, otherwise your generation will go on forever!\n        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN\n        print(text)\n        break\n        texts.append(text)\n    return { "text" : texts, }\npass\n\nfrom datasets import load_dataset\ndataset = load_dataset("yahma/alpaca-cleaned", split = "train")\ndataset = dataset.map(formatting_prompts_func, batched = True,)\n'

In [31]:
import pandas as pd
from transformers import LlamaTokenizer
from datasets import load_dataset

# 定义特殊的 Alpaca 风格的模板
#alpaca_prompt = """Below is a conversation between two people. Write a response to complete the conversation.
alpaca_prompt = """Below is a conversation between two people. Please write a single, non-repetitive response to complete the conversation.

### User:
{}

### Agent:
{}"""

EOS_TOKEN = tokenizer.eos_token  # 必须添加 EOS_TOKEN

def formatting_prompts_func(examples):
    Person1 = examples["User"]
    Person2 = examples["Agent"]
    texts = []
    for Person1, Person2 in zip(Person1, Person2):
        # 必须添加 EOS_TOKEN，否则生成将永远持续下去
        text = alpaca_prompt.format(Person1, Person2) + EOS_TOKEN
        #print(text)
        #break
        texts.append(text)
    return {"text": texts}

# 加载和准备数据集
def prepare_cornell_dataset(file_path):
    # 加载电影台词数据集，并进行处理
    dataset = load_dataset("text", data_files={"train": file_path}, split="train")
    dataset = dataset.map(
        lambda x: {"User": x["text"].split("\t")[0], "Agent": x["text"].split("\t")[1]},
        batched=False
    )
    return dataset

# 调用函数加载数据
dataset = prepare_cornell_dataset("prepared_cornell_dialogs.txt")
'''
print(dataset)

for i in range(5):
    print(f"text: {dataset['text'][i]}")
    print(f"Person 1:{dataset['Person 1'][i]}")
    print(f"Person 2:{dataset['Person 2'][i]}")
'''

# 处理并格式化数据集
tokenized_dataset = dataset.map(formatting_prompts_func, batched=True)


<|end_of_text|>


In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length, #for input
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, #four step refresh model param 
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2): 100%|██████████| 221616/221616 [00:09<00:00, 24257.57 examples/s]
max_steps is given, it will override any value given in num_train_epochs


### Before train

In [32]:
before_ans = " "
#alpaca_prompt = Copied from above
# 启用更快的推理模式
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

# 使用新的模板来定义输入
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.",  # 这是新的 question（对话问题）
            ""  # 这是 answer（答案），留空以让模型生成
        )
    ], 
    return_tensors="pt"
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
with autocast(device_type='cuda'):
    outputs = model.generate(
        **inputs,
        #streamer=text_streamer,
        max_new_tokens=32,
        pad_token_id=tokenizer.eos_token_id,
        use_cache=True
    )
    print(outputs)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(text)
    print("!!!")
    temp = text.split("### User:")[1]
    response = temp.split("### Agent:")[0]
    before_ans = response
    print(response)
#_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
'''
# 生成文本输出
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# 解码输出
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(generated_text)
'''

tensor([[128000,  39314,    374,    264,  10652,   1990,   1403,   1274,     13,
           5321,   3350,    264,   3254,     11,   2536,   5621,   7005,   3486,
           2077,    311,   4686,    279,  10652,    382,  14711,   7508,    220,
             16,    512,   6854,    584,   1304,    420,   4062,     30,    220,
          72523,  20991,    735,  17847,    483,    323,  13929,  56859,    527,
           3515,    459,  17235,  93840,    788,    586,   1464,     12,    709,
            389,    279,  28181,     13,    220,  14077,    382,  14711,   7508,
            220,     17,    512,  40914,     13,    220,    358,   1781,    430,
            596,    279,   2132,    892,    420,   2305,     13,    220,   3639,
            596,    709,   1980,  14711,   7508,    220,     16,    512,     40,
           2846,   2133,    311,   3371,    499,   2555,    430,    358]],
       device='cuda:0')
Below is a conversation between two people. Please write a single, non-repetitive response 

'\n# 生成文本输出\noutputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)\n\n# 解码输出\ngenerated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)\nprint(generated_text)\n'

In [33]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 221,616 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers and Unsloth!


  2%|▏         | 1/60 [00:01<01:10,  1.20s/it]

{'loss': 3.3608, 'grad_norm': 0.0, 'learning_rate': 4e-05, 'epoch': 0.0}


  3%|▎         | 2/60 [00:02<01:00,  1.04s/it]

...

{'loss': 5.3733, 'grad_norm': 0.0, 'learning_rate': 3.636363636363636e-06, 'epoch': 0.0}


100%|██████████| 60/60 [01:01<00:00,  1.06s/it]

{'loss': 3.9994, 'grad_norm': 0.0, 'learning_rate': 0.0, 'epoch': 0.0}


100%|██████████| 60/60 [01:02<00:00,  1.05s/it]

{'train_runtime': 62.9039, 'train_samples_per_second': 7.631, 'train_steps_per_second': 0.954, 'train_loss': 4.342836685975393, 'epoch': 0.0}


In [34]:

torch.cuda.empty_cache()
gc.collect()

33

In [40]:
after_ans = " "
#alpaca_prompt = Copied from above
# 启用更快的推理模式
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

# 使用新的模板来定义输入
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.",  # 这是新的 question（对话问题）
            ""  # 这是 answer（答案），留空以让模型生成
        )
    ], 
    return_tensors="pt"
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
with autocast(device_type='cuda'):
    outputs = model.generate(
        **inputs,
        #streamer=text_streamer,
        max_new_tokens=32,
        use_cache=True
    )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(text)
    print("!!!")
    temp = text.split("### Person 2:")[1]
    response = temp.split("### Person 1:")[0]
    after_ans = response
    print(response)
#_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
'''
# 生成文本输出
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# 解码输出
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(generated_text)
'''

Below is a conversation between two people. Please write a single, non-repetitive response to complete the conversation.

### Person 1:
Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.

### Person 2:
I'm sorry, but I'm not sure I understand.  Who are they, and why is this so important to you?

### Person 1:
R
!!!

I'm sorry, but I'm not sure I understand.  Who are they, and why is this so important to you?




'\n# 生成文本输出\noutputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)\n\n# 解码输出\ngenerated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)\nprint(generated_text)\n'

In [41]:
before_ans = "I don't think I can make it quick.  You know I'm a sucker for public break-ups."
print(f'before_finetune:\n{before_ans}')
print(f'after_finetune:{after_ans}')
print(f'reference:{Question_list[0]}')

before_finetune:
I don't think I can make it quick.  You know I'm a sucker for public break-ups.
after_finetune:
I'm sorry, but I'm not sure I understand.  Who are they, and why is this so important to you?


reference:Well, I thought we'd start with pronunciation, if that's okay with you.


In [42]:
from nltk.translate.gleu_score import sentence_gleu

# 计算 GLEU 分数
gleu_score = sentence_gleu(after_ans, Question_list[0])

print(f"GLEU score: {gleu_score}")

GLEU score: 0.0035971223021582736


In [37]:
import sacrebleu

# 计算 BLEU 分数
bleu = sacrebleu.corpus_bleu(after_ans, Question_list[0])

print(f"BLEU score: {bleu.score}")


BLEU score: 0.0


In [18]:
'''
model.save_pretrained("movielora_model") # Local saving
tokenizer.save_pretrained("movielora_model")
'''

'\nmodel.save_pretrained("movielora_model") # Local saving\ntokenizer.save_pretrained("movielora_model")\n'

In [19]:
'''
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving
'''

'\nmodel.save_pretrained("lora_model") # Local saving\ntokenizer.save_pretrained("lora_model")\n# model.push_to_hub("your_name/lora_model", token = "...") # Online saving\n# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving\n'